In [19]:
import random
from pathlib import Path
import subprocess
import numpy as np
import os


## README ##
## snlc_fit.exe and SALT2mu.exe generate file in the directory they are launched from.


def filter_sn_lightcurve_random(input_file):
    with open(input_file, 'r') as f:
        lines = f.readlines()

    t0 = None
    header = []
    obs_lines = []
    footer = []

    # 1. Parse the file
    for line in lines:
        if line.startswith('PEAKMJD:'):
            t0 = float(line.split()[1])
        
        if line.startswith('OBS:'):
            obs_lines.append(line)
        elif line.startswith('END_PHOTOMETRY:'):
            footer.append(line)
        elif not line.startswith('TRIGGER:'): 
            header.append(line)

    if t0 is None:
        raise ValueError("Could not find PEAKMJD in the header.")

    # 2. Categorize observations into pools
    pool_less_0 = []
    pool_greater_10 = []
    
    for line in obs_lines:
        t1 = float(line.split()[1])
        delta = t1 - t0
        
        if delta < 0:
            pool_less_0.append(line)
        if delta > 10:
            pool_greater_10.append(line)

    # Make sure we have enough data to satisfy the bounds
    if not pool_less_0:
        raise ValueError("No observations found where t1 - t0 < 0.")
    if not pool_greater_10:
        raise ValueError("No observations found where t1 - t0 > 10.")

    # 3. Randomly select the first two required rows
    selected_less_0 = random.choice(pool_less_0)
    selected_greater_10 = random.choice(pool_greater_10)
    
    # Keep track of what we've already picked so we don't duplicate
    already_selected = {selected_less_0, selected_greater_10}

    # 4. Create a pool for the remaining 5 rows, excluding the ones we just picked
    pool_in_range = []
    for line in obs_lines:
        if line in already_selected:
            continue  # Skip rows we already chose
            
        t1 = float(line.split()[1])
        delta = t1 - t0
        
        if -15 <= delta <= 60:
            pool_in_range.append(line)

    if len(pool_in_range) < 5:
        raise ValueError(f"Not enough unique observations in the [-15, 60] range. Found {len(pool_in_range)}, need 5.")

    # Randomly sample exactly 5 unique rows from this range
    selected_in_range = random.sample(pool_in_range, 5)

    # 5. Combine and sort chronologically
    final_selection = [selected_less_0, selected_greater_10] + selected_in_range
    final_selection.sort(key=lambda x: float(x.split()[1]))
    output_file = input_file
    # 6. Write out the new file
    with open(output_file, 'w') as f:
        for line in header:
            if line.startswith('NOBS:'):
                f.write(f"NOBS: {len(final_selection)}\n")
            else:
                f.write(line)
        
        for line in final_selection:
            f.write(line)
            
        for line in footer:
            f.write(line)

def extract_mu_zhel_from_file(file_path):
    """
    Reads a SNANA-format text file from a given path, extracts the MU and zHEL columns, 
    and returns them as numpy arrays.
    
    Parameters:
    file_path (str): The path to the text file.
    
    Returns:
    tuple: Two numpy arrays containing the MU and zHEL values respectively.
    """
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"The file {file_path} was not found.")

    zhel_idx = -1
    mu_idx = -1
    
    zhel_vals = []
    mu_vals = []
    
    # Using 'with open' ensures the file is safely closed after reading
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            
            # Identify column indices from the VARNAMES line
            if line.startswith('VARNAMES:'):
                tokens = line.split()
                zhel_idx = tokens.index('zHEL')
                mu_idx = tokens.index('MU')
                
            # Extract data from the SN rows
            elif line.startswith('SN:'):
                if zhel_idx == -1 or mu_idx == -1:
                    raise ValueError("Encountered data rows before finding VARNAMES header.")
                    
                tokens = line.split()
                
                # Extract and convert to float
                zhel_vals.append(float(tokens[zhel_idx]))
                mu_vals.append(float(tokens[mu_idx]))
                
    return np.array(mu_vals), np.array(zhel_vals)

def process_dat_files(directory_path, action_func):
    """
    Finds every .dat file in the specified directory and applies 
    the action_func to it.
    """
    # Create a Path object for the target directory
    directory = Path(directory_path)
    
    # Check if the directory actually exists to avoid errors
    if not directory.is_dir():
        print(f"Error: The directory '{directory_path}' does not exist.")
        return

    # Use .glob() to find all files ending in .dat
    for file_path in directory.glob('*.dat'):
        # Pass the file path to your custom action function
        action_func(file_path)

def run_exe_and_do_thing(omega_w):
    """
    Launches an executable with arguments, waits for it to finish, 
    and then executes the next steps.
    omega_w is a list of strings, e.g. ["OMEGA_MATTER 0.3", "w 0.7"]
    """
    ## nomi dei files
    nome_run = "TEST1" ## it is like this in the three input files, watch out for it!
    nome_file_input = "sim_SDSS_custom.input"
    nome_file_nml = "snfit_SDSS_custom.nml"
    nome_file_salt2mu = "SALT2mu_DES.input"
    ## paths
    snana_dir = "/home/ubuntu/SNANA"
    bin_dir = f"{snana_dir}/SNDIR/bin"
    snlc_sim_path = f"{bin_dir}/snlc_sim.exe"
    snlc_fit_path = f"{bin_dir}/snlc_fit.exe"
    salt2mu_path = f"{bin_dir}/SALT2mu.exe"
    ## dir where we save snlc_sim.exe output .dat files 
    sim_dir_path = f"{snana_dir}/SNROOT/SIM/{nome_run}"
    ## dir of the input files
    snlc_sim_input = f"{snana_dir}/custom_input_files/{nome_file_input}"
    snlc_fit_input = f"{snana_dir}/custom_input_files/{nome_file_nml}"
    salt2mu_input = f"{snana_dir}/custom_input_files/{nome_file_salt2mu}"

    sim_command = [snlc_sim_path, snlc_sim_input] + omega_w
    fit_command = [snlc_fit_path, snlc_fit_input]
    salt2mu_command = [salt2mu_path, salt2mu_input]

    result = subprocess.run(sim_command, capture_output=True, text=True, check=True)
    
    print("The .exe finished running successfully!")
    print(f"Here is what the .exe output: {result.stdout}")
    print("cleaning")
    process_dat_files(sim_dir_path, filter_sn_lightcurve_random)
    print("extracted 7 points from dat files")
    print(f"Launching: {snlc_fit_path}")
    result = subprocess.run(fit_command, capture_output=True, text=True, check=True)
    print(result.stdout)
    print("fit done, output should be in where u launched this script")
    salt2mu_output = f"{snana_dir}/scripts/SALT2mu_{nome_run}.FITRES"
    result = subprocess.run(salt2mu_command, capture_output=True, text=True, check=True)
    print(f"Salt2mu output: {result.stdout}")
    a, b = extract_mu_zhel_from_file(salt2mu_output)
    return a, b





In [26]:
a, b = run_exe_and_do_thing(["OMEGA_MATTER", "0.3"])
print(a)
print(b)

The .exe finished running successfully!
Here is what the .exe output: 
 Full command: /home/ubuntu/SNANA/SNDIR/bin/snlc_sim.exe /home/ubuntu/SNANA/custom_input_files/sim_SDSS_custom.input OMEGA_MATTER 0.3 

 SNANA_VERSION: 96036915



 ****************************************************************** 
   Begin execution of snlc_sim.exe    
 SNDATA_ROOT = /home/ubuntu/SNANA/SNROOT 
 SNANA_DIR   = /home/ubuntu/SNANA/SNDIR 


   sizeof(INPUTS) =   5.247 MB 
   sizeof(GENLC)  =  17.393 MB 

  init_SNDATA_GLOBAL: 
 --------------------------------------------------------
  Initialize NstringMatch for sim-input file
  Read 87 words from user input file 1: 
	 /home/ubuntu/SNANA/custom_input_files/sim_SDSS_custom.input 

 ****************************************************************** 
   init_HzFUN_INFO  
	 H0         = 70.00      # km/s/Mpc 
	 OM, OL, Ok = 0.30000, 0.68500, 0.01500 
	 w0, wa     = -1.000,  0.000 

 ****************************************************************** 
    p